# general checks for these cases (what variables to make)
- 1. boundaries
- 2. window state: when you MUST know about the current window
- 3. best-so-far
- 4. constraints/targets

# chunking

In [ ]:
# demo for chunking
# can't run
for i in range(0,n,k):
    chunk = arr[i:i+k]

In [ ]:
from typing import List

def chunk(text: str, size=50, overlap=10):
    out, i = [], 0
    while i < len(text):
        out.append(text[i:i+size])  #text[0,50]->out
        i += max(1, size - overlap) # i=0 -> max(1, 50-40) = 40
    return out
    
chunked_list = chunk("A"*120, size=50, overlap=10)
print(len(chunked_list[2]))


# sliding window
clues are often 👇🏻
- 1. subarray/substring
- 2. longest/shortest
- 3. at most/at least
- 4. continuous segment/consecutive (12 values for example)

In [ ]:
# demo code for sliding window
# can't run 

left = 0
for right in range(n):
    # expand
    while invalid:
        left += 1

# inverted index
This is the foundational structure behind search engines, keyword recall, RAG systems, and hybrid retrieval.

In [ ]:
from collections import defaultdict

docs = {
    1: "refund policy and receipt",
    2: "how to request a refund",
    3: "lost receipt replacement"
}

index = defaultdict(set)
print(list(index))  #[]

for doc_id, text in docs.items():   #kv pairs
    for word in text.lower().split():
        index[word].add(doc_id)

print(index["refund"])


[]
{1, 2}


# class cases

## RAG pipeline

score

In [ ]:
valid_score = [s for s in score if 0 <= s <= 1] # get pure score
scored = [(c, mock_similarity(query, c.text)) for c in chunks]  #get (related_part_in_doc, similarity_score) pairs

In [1]:
class Retriever:
    def retrieve(self, query):
        return [f"doc about {query} (score=0.82)"]

class Generator:
    def generate(self, query, docs):
        return f"Answer to '{query}' using {len(docs)} docs."

class RAGPipeline:
    def __init__(self, retriever, generator):
        self.retriever = retriever
        self.generator = generator

    def run(self, query):
        docs = self.retriever.retrieve(query)
        return self.generator.generate(query, docs)

pipe = RAGPipeline(Retriever(), Generator())
print(pipe.run("refund policy"))

Answer to 'refund policy' using 1 docs.


chunking with overlap

In [ ]:
# eg. for a 50,000 chinese character document
# can't run

from dataclasses import dataclass
from typing import List, Dict, Any

@dataclass
class Chunk:
    chunk_id: str
    text: str
    meta: Dict[str, Any]  # e.g. {"section": "4.2", "page": 12}

def chunk_text_by_chars(
    doc_text: str,
    chunk_size: int = 600,
    overlap: int = 120,
    doc_id: str = "doc1"
) -> List[Chunk]:
    """
    Character-based chunking (works well for Chinese).
    """
    assert 0 <= overlap < chunk_size        # 🤩use of assert here
    chunks = []
    start = 0
    idx = 0
    while start < len(doc_text):
        end = min(len(doc_text), start + chunk_size)
        text = doc_text[start:end].strip()
        if text:
            chunks.append(Chunk(
                chunk_id=f"{doc_id}_c{idx}",
                text=text,
                meta={"doc_id": doc_id, "start": start, "end": end}
            ))
            idx += 1
        start += (chunk_size - overlap)
    return chunks

# Example
doc = "（假设这里是很长的中文文档...）" * 5000
chunks = chunk_text_by_chars(doc, chunk_size=600, overlap=120, doc_id="manual_cn")
print(len(chunks), chunks[0].chunk_id, len(chunks[0].text))

178 manual_cn_c0 600


List[Chunk] meaning visually 👇🏻

In [ ]:
chunks: List[Chunk] = [
    Chunk("c0", "a", 0, 1),
    Chunk("c1", "b", 1, 2),
]

## agent

In [ ]:
# crewai example

searcher = Agent(
    role='Content Searcher',
    goal='Search for online content about {topic}',
    backstory='''
        You will be working on creating articles for LinkedIn about {topic}. 
        You will conduct research online, collect and group information. 
        Your work will serve as a basis for the Content Writer.
    ''',
    llm=my_llm,
    verbose=True,       # if detailed logging needed
    allow_delegate=False,
    tools=[search_tool]
)

aside from that, we need to define Task for each agent

In [ ]:
# eg, for the Search Agent 
search = Task(
    description='''
        1. Give priority to the most recent trends, key figures, 
           and breaking news related to {topic}.
        2. Determine the target audience, taking into account 
           their interests and challenges.
        3. Incorporate SEO keywords and data from credible sources.
    ''',
    agent=searcher,
    expected_output='A comprehensive trend plan focused on {topic}, incorporating top SEO keywords and the most recent news.'
)

in multi-agent mode, we shall ensemble the crew at the end

## multiagent & crew

In [ ]:
crew = Crew(
    agents=[searcher, writer, editor],
    tasks=[search, write, review],
    verbose=1,
    process=Process.sequential
    memory=True      # enable crew memory
)

# fastapi

In [ ]:
app.post("/chat/data_sql_chat",
             tags=["Chat"],
             summary="与元芳直接对话返回SQL(通过多段请求)",
             )(data_sql_chat)

    app.post("/chat/chat",
             tags=["Chat"],
             summary="与llm模型对话(通过LLMChain)",
             )(chat)